In [17]:
import gspread

gc = gspread.oauth()

In [18]:
wb_deepwork = gc.open('Deep Work')
wb_tracker = gc.open('Personal Tracking')
sh_focus = wb_deepwork.worksheet('Focus')
cells_focus = sh_focus.get_all_records()

In [19]:
def get_cells(gc, workbook_name, worksheet_name):
    workbook = gc.open(workbook_name)
    worksheet = workbook.worksheet(worksheet_name)
    cells = worksheet.get_all_records()

    return cells

In [20]:
cells_focus = get_cells(gc, workbook_name='Deep Work', worksheet_name='Focus')

In [21]:
import pandas as pd, numpy as np
from quant_helper.data_cleaning import cols_to_datetime, format_datetime_index

df_focus = (
    pd.DataFrame(cells_focus)
    .pipe(cols_to_datetime, cols=['Start', 'Stop'])
    .assign(duration = lambda x: (x['Stop'] - x['Start']).divide(pd.Timedelta(hours=1)))
    .dropna(axis=0, subset=['Type'])
)

In [22]:
df_focus.tail(20)

,Task,Start,Stop,Type,Quality,Notes,Distractions,Date,Hours,Checks/Hr,MA(10),,duration
198,Set up VPS,2020-07-20 14:47:53,2020-07-20 15:55:03,Deep,7,lots of waiting for things to install,3,,,,,,1.119444
199,Set up data store/work on strategy report temp...,2020-07-20 16:45:09,2020-07-20 17:56:23,Deep,8,,2,,,,,,1.187222
200,Clear out daily tasks,2020-07-21 07:50:00,2020-07-21 08:16:00,Shallow,9,,1,,,,,,0.433333
201,Work on Strategy Report Template,2020-07-21 08:16:00,2020-07-21 09:01:07,Deep,10,,0,,,,,,0.751944
202,AWEV Demo Meeting,2020-07-21 09:01:07,2020-07-21 09:46:11,Meeting,7,,2,,,,,,0.751111
203,Prepare for MPP Demo,2020-07-21 09:46:11,2020-07-21 10:01:28,Shallow,8,,1,,,,,,0.254722
204,MPP Training Demo,2020-07-21 10:01:50,2020-07-21 12:00:00,Meeting,10,,0,,,,,,1.969444
205,Lunch/Articles,2020-07-21 12:00:00,2020-07-21 13:00:00,Education,8,,2,,,,,,1.000000
206,Set up data store,2020-07-21 14:49:00,2020-07-21 15:13:02,Deep,10,,0,,,,,,0.400556
207,Set up data store,2020-07-21 15:27:54,2020-07-21 16:25:21,Deep,9,,1,,,,,,0.957500


In [23]:
import matplotlib.pyplot as plt

summary_focus_daily = (
    df_focus
    .groupby([df_focus['Start'].dt.date, 'Type'])
    ['duration']
    .sum()
    .unstack()
    .pipe(format_datetime_index)
    .resample('D')
    .last()
    .fillna(0)
    .loc['7/2020':]
)

In [24]:
summary_focus_daily

Type,Deep,Education,Meeting,None,Shallow
date,,,,,
2020-07-01,1.750000,0.000000,0.000000,0.000000,2.733333
2020-07-02,0.000000,0.000000,0.000000,0.000000,0.833333
2020-07-03,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-04,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-05,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-06,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-07,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-08,0.000000,0.000000,0.000000,0.000000,0.000000
2020-07-09,1.433889,0.000000,0.000000,0.205833,4.546389


In [25]:
# def convert_col_dtypes(df, cols, dtypes):
#     for col, dtype in zip(cols, dtypes):
#         df[col] = df[col].astype(dtype)

#     return df

from quant_helper.data_cleaning import convert_col_dtypes

In [26]:
summary_focus_daily_formatted = (
    summary_focus_daily
    .reset_index()
    .pipe(convert_col_dtypes, ['date'], [str])
)

In [27]:
summary_focus_daily_formatted

Type,date,Deep,Education,Meeting,None,Shallow
0,2020-07-01,1.750000,0.000000,0.000000,0.000000,2.733333
1,2020-07-02,0.000000,0.000000,0.000000,0.000000,0.833333
2,2020-07-03,0.000000,0.000000,0.000000,0.000000,0.000000
3,2020-07-04,0.000000,0.000000,0.000000,0.000000,0.000000
4,2020-07-05,0.000000,0.000000,0.000000,0.000000,0.000000
5,2020-07-06,0.000000,0.000000,0.000000,0.000000,0.000000
6,2020-07-07,0.000000,0.000000,0.000000,0.000000,0.000000
7,2020-07-08,0.000000,0.000000,0.000000,0.000000,0.000000
8,2020-07-09,1.433889,0.000000,0.000000,0.205833,4.546389
9,2020-07-10,1.275833,0.000000,0.750000,0.000000,2.562500


In [28]:
# def update_gsheet_pandas(worksheet, df):
#     worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
from modern_renaissance.gspread import update_gsheet_pandas

In [29]:
sh_work = wb_tracker.worksheet('Work')

In [30]:
update_gsheet_pandas(sh_work, summary_focus_daily_formatted)

# Logging Stuff

In [31]:
from discord_webhook import DiscordWebhook, DiscordEmbed